### Fund holdings and MSCI data - Excluding Index funds and ETF's
* This notebook includes the construction/ analysis of final fund holdings and MSCI ratings dataframe excluding index funds and ETF's.

---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Load fund and ESG data

In [2]:
msci_data = pd.read_csv('../data/msci_data_subset.csv')

In [3]:
msci_data = msci_data.dropna()

In [4]:
fund_header = pd.read_csv('../data/fund_hdr_table.csv')
holdings_data = pd.read_csv('../data/holdings_data.csv', dtype={'maturity_dt': str})

In [5]:
fund_style_equity_only = pd.read_csv('../data/fund_style_equity_only.csv')

In [6]:
check_list = ["SRI", "social", "ESG", "green", "sustain", "environ", "impact", "responsible", "clean", "renewable"]

In [7]:
'|'.join(check_list)

'SRI|social|ESG|green|sustain|environ|impact|responsible|clean|renewable'

In [8]:
listed_esg_fund_names = fund_header[fund_header.fund_name.str.contains('|'.join(check_list), case=False, regex=True)]

In [9]:
len(listed_esg_fund_names)

1783

### Filter out index funds and ETF's

In [10]:
funds_excluding_index_etfs = pd.read_csv('funds_excluding_index_etf_etn.csv')

In [11]:
funds_excluding_index_etfs

,fund_name
0,Spectra Fund: Spectra Green Fund; Class N Shares
1,Alger Institutional Funds: Alger Green Institu...
2,"American Century Mutual Funds, Inc: Sustainabl..."
3,AMG Funds I: AMG Boston Common Global Impact F...
4,Forum Funds II: Baywood SociallyResponsible Fu...
...,...
100,Fidelity Charles Street Trust: Fidelity Sustai...
101,EQ Advisors Trust: EQ/AB Sustainable US Themat...
102,Fidelity Summer Street Trust: Fidelity SAI Sus...
103,Fidelity Summer Street Trust: Fidelity SAI Sus...


In [12]:
listed_esg_fund_names = listed_esg_fund_names.merge(funds_excluding_index_etfs, how='inner', on='fund_name', validate='m:1')

In [13]:
len(listed_esg_fund_names)

105

In [14]:
# fund style/strategy included in research
styles = ['growth', 'income', 'small', 'mid']

In [15]:
fund_style_subset = fund_style_equity_only[fund_style_equity_only.lipper_obj_name.str.contains('|'.join(styles), case=False, regex=True)]

In [16]:
len(fund_style_subset)

61198

In [17]:
# get domestic funds with 'ED', 'ED' being equity,domestic
domestic_fund_style_subset = fund_style_subset[fund_style_subset['crsp_obj_cd'].str.contains('ED')]

In [18]:
len(domestic_fund_style_subset)

57495

In [19]:
domestic_fund_style_subset['lipper_obj_name'].value_counts()

GROWTH FUNDS               15076
GROWTH & INCOME FUNDS      11855
Growth Funds                7440
MID-CAP FUNDS               5761
SMALL-CAP FUNDS             5294
Growth and Income Funds     4329
Small-Cap Funds             3383
Mid-Cap Funds               2414
EQUITY INCOME FUNDS         1313
Equity Income Funds          630
Name: lipper_obj_name, dtype: int64

In [20]:
# map fund style to fund name on crsp_fundno

In [21]:
fund_name_style_table = domestic_fund_style_subset.merge(listed_esg_fund_names, how='inner', on='crsp_fundno', validate='m:1')

In [22]:
fund_name_style_table.shape

(266, 38)

In [23]:
fund_name_style_table['lipper_obj_name'].value_counts()

GROWTH FUNDS               133
GROWTH & INCOME FUNDS       52
Growth Funds                32
MID-CAP FUNDS               16
SMALL-CAP FUNDS             12
Small-Cap Funds              7
EQUITY INCOME FUNDS          7
Growth and Income Funds      5
Mid-Cap Funds                2
Name: lipper_obj_name, dtype: int64

In [24]:
len(fund_name_style_table.crsp_fundno.unique())

105

### Funds with each of it's classes

In [25]:
fund_name_style_table.groupby(['crsp_fundno', 'lipper_class_name'])['lipper_class_name'].nunique().to_frame()

lipper_class_name
crsp_fundno lipper_class_name                                
3946        Large-Cap Growth Funds                          1
            Multi-Cap Growth Funds                          1
3955        Multi-Cap Growth Funds                          1
5031        Large-Cap Core Funds                            1
6718        Mid-Cap Core Funds                              1
...                                                       ...
99620       Mixed-Asset Trgt Alloc Gro Fds                  1
99632       Multi-Cap Core Funds                            1
99856       Mid-Cap Core Funds                              1
99860       Mid-Cap Core Funds                              1
99861       Mid-Cap Core Funds                              1

[193 rows x 1 columns]

### Fund name and holdings data merged

In [26]:
fund_name_style_table = fund_name_style_table.loc[:, ['crsp_fundno', 'fund_name', 'crsp_portno', 'crsp_obj_cd', 
                                     'lipper_class_name', 'lipper_obj_name', 'lipper_asset_cd']]

In [27]:
fund_name_style_table['lipper_obj_name'] = fund_name_style_table['lipper_obj_name'].str.lower()

In [28]:
fund_name_style_table.loc[fund_name_style_table['lipper_obj_name']=='growth & income funds', 'lipper_obj_name'] = 'growth and income funds'

In [29]:
fund_name_style_table = fund_name_style_table.drop_duplicates().dropna(subset='crsp_portno')

In [30]:
fund_name_style_table.lipper_obj_name.unique()

array(['growth funds', 'growth and income funds', 'small-cap funds',
       'mid-cap funds', 'equity income funds'], dtype=object)

In [31]:
fund_name_style_table = fund_name_style_table.drop_duplicates(subset=['crsp_portno'])

In [32]:
funds_excluding_index_etfs = pd.read_csv('funds_excluding_index_etf_etn.csv')

In [33]:
funds_excluding_index_etfs

,fund_name
0,Spectra Fund: Spectra Green Fund; Class N Shares
1,Alger Institutional Funds: Alger Green Institu...
2,"American Century Mutual Funds, Inc: Sustainabl..."
3,AMG Funds I: AMG Boston Common Global Impact F...
4,Forum Funds II: Baywood SociallyResponsible Fu...
...,...
100,Fidelity Charles Street Trust: Fidelity Sustai...
101,EQ Advisors Trust: EQ/AB Sustainable US Themat...
102,Fidelity Summer Street Trust: Fidelity SAI Sus...
103,Fidelity Summer Street Trust: Fidelity SAI Sus...


In [34]:
fund_name_style_table = fund_name_style_table.merge(funds_excluding_index_etfs, how='inner', on='fund_name', validate='1:1')

In [35]:
len(fund_name_style_table)

105

In [36]:
fund_and_holdings_table = fund_name_style_table.merge(holdings_data, how='inner', on='crsp_portno')

In [37]:
#here I have 19712 unique securities

In [38]:
len(fund_and_holdings_table.security_name.unique())

11179

In [39]:
to_drop = 'Valued Advisers Trust: Green Owl Intrinsic Value Fund'

In [40]:
fund_and_holdings_table = fund_and_holdings_table.drop(
    fund_and_holdings_table[fund_and_holdings_table['fund_name'] == to_drop].index
)

## I identify 105 funds

In [41]:
fund_and_holdings_table.loc[:, ['crsp_fundno', 'fund_name', 'lipper_class_name']].drop_duplicates().to_excel('active_fund_names_excl_index_etf.xlsx', index=False)

In [42]:
fund_and_holdings_table.groupby(['crsp_fundno', 'lipper_class_name'])['lipper_class_name'].nunique().to_frame()

,,lipper_class_name
crsp_fundno,lipper_class_name,
3946,Large-Cap Growth Funds,1
3955,Multi-Cap Growth Funds,1
5031,Large-Cap Core Funds,1
6718,Multi-Cap Growth Funds,1
6876,Multi-Cap Value Funds,1
...,...,...
99620,Mixed-Asset Trgt Alloc Gro Fds,1
99632,Multi-Cap Core Funds,1
99856,Mid-Cap Core Funds,1


In [43]:
final_fund_holdings_table_short = fund_and_holdings_table.loc[:, ['crsp_fundno', 'report_dt', 'nbr_shares', 'security_name', 'cusip', 'ticker']]

In [44]:
final_fund_holdings_table_short

,crsp_fundno,report_dt,nbr_shares,security_name,cusip,ticker
0,3946,2009/10/31,6365,APPLE INC,03783310,AAPL
1,3946,2009/10/31,38330,MICROSOFT CORP,59491810,MSFT
2,3946,2009/10/31,1370,GOOGLE INC,38259P50,GOOG
3,3946,2009/10/31,13660,WAL MART STORES INC,93114210,WMT
4,3946,2009/10/31,13250,TARGET CORP,87612E10,TGT
...,...,...,...,...,...,...
1378076,99861,2022/05/31,19,GENERAC HOLDINGS INC,36873610,GNRC
1378077,99861,2022/05/31,31,AMERICAN WATER WORKS CO INC,03042010,AWK
1378078,99861,2022/05/31,4354,Fidelity Cash Central Fund,NaN,LP4020
1378079,99861,2022/05/31,44,GUARDANT HEALTH INC,40131M10,GH


In [45]:
final_fund_holdings_table_short['year_month'] = pd.to_datetime([x[:7] for x in final_fund_holdings_table_short.report_dt])

In [46]:
final_fund_holdings_table_short['year_month'] = final_fund_holdings_table_short['year_month'].dt.to_period('M')

In [47]:
final_fund_holdings_table_short = final_fund_holdings_table_short.drop('report_dt', axis=1)

In [48]:
final_fund_holdings_table_short

,crsp_fundno,nbr_shares,security_name,cusip,ticker,year_month
0,3946,6365,APPLE INC,03783310,AAPL,2009-10
1,3946,38330,MICROSOFT CORP,59491810,MSFT,2009-10
2,3946,1370,GOOGLE INC,38259P50,GOOG,2009-10
3,3946,13660,WAL MART STORES INC,93114210,WMT,2009-10
4,3946,13250,TARGET CORP,87612E10,TGT,2009-10
...,...,...,...,...,...,...
1378076,99861,19,GENERAC HOLDINGS INC,36873610,GNRC,2022-05
1378077,99861,31,AMERICAN WATER WORKS CO INC,03042010,AWK,2022-05
1378078,99861,4354,Fidelity Cash Central Fund,NaN,LP4020,2022-05
1378079,99861,44,GUARDANT HEALTH INC,40131M10,GH,2022-05


### Load input to map cusip to isin

In [49]:
sec = pd.read_csv(r"C:\Users\johnd\PycharmProjects\pythonproject\Basecamp\sec_header.csv")

In [50]:
sec = sec.drop_duplicates(subset='scusip')

In [51]:
sec

,scusip,tic,isin
0,000032102,AE.2,NaN
1,000165100,AMFD.,US0001651001
2,000354100,ANTQ,US0003541002
3,000361105,AIR,US0003611052
4,000781104,ABSI.1,US0007811047
...,...,...,...
47450,67013H114,NOW.WT,CA67013H1148
47451,37428A103,GET,US37428A1034
47452,74907L102,QNRX,US74907L1026
47453,67578E100,ODDAF,CA67578E1007


In [52]:
final_fund_holdings_table_short.cusip.drop_duplicates().dropna().to_excel('cusip_8.xlsx', index=False)

In [53]:
cusip_9_digit = pd.read_csv('../data/cusip_9_overall_redone.csv')

In [54]:
cusip_9_digit = cusip_9_digit.rename(columns={'cusip':'cusip_9'})

In [55]:
cusip_9_digit['cusip_8'] = [x[:8] for x in cusip_9_digit['cusip_9']]

In [56]:
cusip_and_isins = cusip_9_digit.merge(sec, how='inner', left_on='cusip_9', right_on='scusip', validate='1:1')

In [57]:
cusip_and_isins

,cusip_9,cusip_8,scusip,tic,isin
0,000307108,00030710,000307108,AACH,US0003071083
1,000360206,00036020,000360206,AAON,US0003602069
2,000361105,00036110,000361105,AIR,US0003611052
3,000375204,00037520,000375204,ABB,US0003752047
4,000380204,00038020,000380204,ABCM,US0003802040
...,...,...,...,...,...
6594,Y8564W103,Y8564W10,Y8564W103,TK,MHY8564W1030
6595,Y8565N300,Y8565N30,Y8565N300,TNK,MHY8565N3002
6596,Y8897Y180,Y8897Y18,Y8897Y180,TOPS,MHY8897Y1804
6597,Y93691106,Y9369110,Y93691106,VRGY,SG9999002885


In [58]:
# merge above table with msci data

In [59]:
msci_isin_mapped = msci_data.merge(cusip_and_isins, how='inner', left_on='ISIN', right_on='isin', validate='m:1')

In [60]:
len(msci_isin_mapped)

60778

In [61]:
len(msci_isin_mapped.drop_duplicates(subset='isin'))

1038

In [62]:
msci_isin_mapped

,Analysis Date,ISIN,Asset Name,Company Rating,Industry Adjusted Score,cusip_9,cusip_8,scusip,tic,isin
0,2013-12-31,CA0679011084,BARRICK GOLD CORP,B,2.3,067901108,06790110,067901108,GOLD,CA0679011084
1,2014-01-31,CA0679011084,BARRICK GOLD CORP,B,2.3,067901108,06790110,067901108,GOLD,CA0679011084
2,2014-02-28,CA0679011084,BARRICK GOLD CORP,B,2.3,067901108,06790110,067901108,GOLD,CA0679011084
3,2014-03-31,CA0679011084,BARRICK GOLD CORP,B,2.3,067901108,06790110,067901108,GOLD,CA0679011084
4,2014-04-30,CA0679011084,BARRICK GOLD CORP,B,2.4,067901108,06790110,067901108,GOLD,CA0679011084
...,...,...,...,...,...,...,...,...,...,...
60773,2020-12-31,US29786A1060,ETSY INC,BBB,5.5,29786A106,29786A10,29786A106,ETSY,US29786A1060
60774,2020-12-31,CA33767E2024,FIRSTSERVICE CORP,A,6.9,33767E202,33767E20,33767E202,FSV,CA33767E2024
60775,2020-12-31,JE00BYSS4X48,NOVOCURE LTD,BBB,4.7,G6674U108,G6674U10,G6674U108,NVCR,JE00BYSS4X48
60776,2020-12-31,US1468691027,CARVANA CO,B,2.4,146869102,14686910,146869102,CVNA,US1468691027


In [63]:
msci_isin_mapped['year_month'] = pd.to_datetime([x[:7] for x in msci_isin_mapped['Analysis Date']])

In [64]:
msci_isin_mapped['year_month'] = msci_isin_mapped['year_month'].dt.to_period('M')

In [65]:
msci_isin_mapped

,Analysis Date,ISIN,Asset Name,Company Rating,Industry Adjusted Score,cusip_9,cusip_8,scusip,tic,isin,year_month
0,2013-12-31,CA0679011084,BARRICK GOLD CORP,B,2.3,067901108,06790110,067901108,GOLD,CA0679011084,2013-12
1,2014-01-31,CA0679011084,BARRICK GOLD CORP,B,2.3,067901108,06790110,067901108,GOLD,CA0679011084,2014-01
2,2014-02-28,CA0679011084,BARRICK GOLD CORP,B,2.3,067901108,06790110,067901108,GOLD,CA0679011084,2014-02
3,2014-03-31,CA0679011084,BARRICK GOLD CORP,B,2.3,067901108,06790110,067901108,GOLD,CA0679011084,2014-03
4,2014-04-30,CA0679011084,BARRICK GOLD CORP,B,2.4,067901108,06790110,067901108,GOLD,CA0679011084,2014-04
...,...,...,...,...,...,...,...,...,...,...,...
60773,2020-12-31,US29786A1060,ETSY INC,BBB,5.5,29786A106,29786A10,29786A106,ETSY,US29786A1060,2020-12
60774,2020-12-31,CA33767E2024,FIRSTSERVICE CORP,A,6.9,33767E202,33767E20,33767E202,FSV,CA33767E2024,2020-12
60775,2020-12-31,JE00BYSS4X48,NOVOCURE LTD,BBB,4.7,G6674U108,G6674U10,G6674U108,NVCR,JE00BYSS4X48,2020-12
60776,2020-12-31,US1468691027,CARVANA CO,B,2.4,146869102,14686910,146869102,CVNA,US1468691027,2020-12


In [66]:
# idea is to match to fund table to get number of shares held

In [67]:
companies_in_both_frames = (final_fund_holdings_table_short.drop_duplicates(subset='cusip').merge(msci_isin_mapped.drop_duplicates(subset='isin'),
                                                                      left_on='cusip', right_on='cusip_8', validate='1:1'))

In [68]:
# identify companies from fund_holdings table for which you have msci ratings (basically from msci_isin_mapped table)

In [69]:
required_fund_holdings_table = final_fund_holdings_table_short.set_index('cusip').loc[companies_in_both_frames['cusip']]

In [70]:
required_fund_holdings_table = required_fund_holdings_table.reset_index().sort_values(['crsp_fundno', 'cusip'])

In [71]:
required_fund_holdings_table

,cusip,crsp_fundno,nbr_shares,security_name,ticker,year_month
30531,00282410,3946,7800,ABBOTT LABORATORIES,ABT,2009-10
30532,00282410,3946,8450,ABBOTT LABORATORIES,ABT,2010-01
30533,00282410,3946,9345,ABBOTT LABORATORIES,ABT,2010-04
30534,00282410,3946,9345,ABBOTT LABORATORIES,ABT,2010-06
30535,00282410,3946,9345,ABBOTT LABORATORIES,ABT,2010-07
...,...,...,...,...,...,...
115450,98978V10,99861,59,ZOETIS INC,ZTS,2022-05
656704,F5848511,99861,14,LVMH Moet Hennessy Louis Vuitton SE ORD,LVMH,2022-05
180555,G1151C10,99861,53,ACCENTURE PLC IRELAND,ACN,2022-05
138583,G5150210,99861,131,JOHNSON CONTROLS INTL PLC,JCI,2022-05


### Drop negative holdings.

In [72]:
required_fund_holdings_table = required_fund_holdings_table[~required_fund_holdings_table.nbr_shares<0]

In [73]:
agg_holdings = required_fund_holdings_table.groupby(['cusip', 'year_month'])['nbr_shares'].sum().to_frame().reset_index()

In [74]:
agg_holdings

,cusip,year_month,nbr_shares
0,00101J10,2012-10,172179
1,00101J10,2012-11,89472
2,00101J10,2012-12,224694
3,00101J10,2013-01,96886
4,00101J10,2013-02,79250
...,...,...,...
111162,Y2573F10,2022-01,617780
111163,Y2573F10,2022-02,269802
111164,Y2573F10,2022-03,102065
111165,Y2573F10,2022-04,478828


In [75]:
# these cusips have just one observation
for x in agg_holdings['cusip'].drop_duplicates():
    table_1 = agg_holdings.set_index('cusip').loc[x]
    if type(table_1) == pd.core.series.Series:
        print(x)

11282X10
45303840
55270410
69790010
L6388F12


### Do a forward fill to obtain complete time-series of fund holdings data

In [76]:
%%time
esg_fund_time_series = []
for x in agg_holdings['cusip'].drop_duplicates():
    table_1 = agg_holdings.set_index('cusip').loc[x]
    if type(table_1) != pd.core.series.Series:
        table_1 = table_1.drop_duplicates(subset='year_month').reset_index().set_index('year_month').resample('M').ffill()
        esg_fund_time_series.append(table_1)
    print('completed',len(esg_fund_time_series))

completed 1
completed 2
completed 3
completed 4
completed 5
completed 6
completed 7
completed 8
completed 9
completed 10
completed 11
completed 12
completed 13
completed 14
completed 15
completed 16
completed 17
completed 18
completed 19
completed 20
completed 21
completed 22
completed 23
completed 24
completed 25
completed 26
completed 27
completed 28
completed 29
completed 30
completed 31
completed 32
completed 33
completed 34
completed 35
completed 36
completed 37
completed 38
completed 39
completed 40
completed 41
completed 42
completed 43
completed 44
completed 45
completed 46
completed 47
completed 48
completed 49
completed 50
completed 51
completed 52
completed 53
completed 54
completed 55
completed 56
completed 57
completed 58
completed 59
completed 60
completed 61
completed 62
completed 63
completed 64
completed 65
completed 66
completed 67
completed 68
completed 69
completed 70
completed 71
completed 72
completed 73
completed 74
completed 75
completed 76
completed 77
complete

completed 594
completed 595
completed 596
completed 597
completed 598
completed 599
completed 600
completed 601
completed 602
completed 603
completed 604
completed 605
completed 606
completed 607
completed 608
completed 609
completed 610
completed 611
completed 612
completed 613
completed 614
completed 615
completed 616
completed 617
completed 618
completed 619
completed 620
completed 621
completed 622
completed 623
completed 624
completed 625
completed 626
completed 627
completed 628
completed 629
completed 630
completed 631
completed 632
completed 633
completed 634
completed 635
completed 636
completed 637
completed 638
completed 639
completed 640
completed 641
completed 642
completed 643
completed 644
completed 645
completed 646
completed 647
completed 648
completed 649
completed 650
completed 651
completed 652
completed 653
completed 654
completed 655
completed 656
completed 657
completed 658
completed 659
completed 660
completed 661
completed 662
completed 663
completed 664
comple

In [77]:
agg_holdings = pd.concat(esg_fund_time_series).reset_index()

In [78]:
agg_holdings

,year_month,cusip,nbr_shares
0,2012-10,00101J10,172179
1,2012-11,00101J10,89472
2,2012-12,00101J10,224694
3,2013-01,00101J10,96886
4,2013-02,00101J10,79250
...,...,...,...
128629,2022-01,Y2573F10,617780
128630,2022-02,Y2573F10,269802
128631,2022-03,Y2573F10,102065
128632,2022-04,Y2573F10,478828


In [79]:
required_msci_isin_mapped = msci_isin_mapped.loc[:,['year_month','cusip_8', 'ISIN', 'Asset Name', 'Company Rating', 'Industry Adjusted Score']].sort_values(['cusip_8', 'year_month'])

In [80]:
required_msci_isin_mapped = required_msci_isin_mapped.groupby(['cusip_8', 'year_month']).value_counts().to_frame().drop(0, axis=1).reset_index()

In [81]:
required_msci_isin_mapped

,cusip_8,year_month,ISIN,Asset Name,Company Rating,Industry Adjusted Score
0,00101J10,2013-12,US00101J1060,ADT CORP,BBB,4.4
1,00101J10,2014-01,US00101J1060,ADT CORP,BBB,4.4
2,00101J10,2014-02,US00101J1060,ADT CORP,BBB,4.4
3,00101J10,2014-03,US00101J1060,ADT CORP,BBB,4.4
4,00101J10,2014-04,US00101J1060,ADT CORP,BBB,4.4
...,...,...,...,...,...,...
60773,Y2573F10,2020-01,SG9999000020,FLEX LTD,A,6.1
60774,Y2573F10,2020-02,SG9999000020,FLEX LTD,A,6.9
60775,Y2573F10,2020-03,SG9999000020,FLEX LTD,A,6.9
60776,Y2573F10,2020-04,SG9999000020,FLEX LTD,A,6.9


In [82]:
len([x for x in agg_holdings['cusip'].drop_duplicates()])

967

In [83]:
final_table_for_stata = agg_holdings.merge(required_msci_isin_mapped, how='inner', left_on=['cusip', 'year_month'], right_on=['cusip_8', 'year_month'], validate='1:1')

In [84]:
final_table_for_stata

,year_month,cusip,nbr_shares,cusip_8,ISIN,Asset Name,Company Rating,Industry Adjusted Score
0,2013-12,00101J10,231801,00101J10,US00101J1060,ADT CORP,BBB,4.4
1,2014-01,00101J10,152417,00101J10,US00101J1060,ADT CORP,BBB,4.4
2,2014-02,00101J10,136968,00101J10,US00101J1060,ADT CORP,BBB,4.4
3,2014-03,00101J10,59068,00101J10,US00101J1060,ADT CORP,BBB,4.4
4,2014-04,00101J10,17834,00101J10,US00101J1060,ADT CORP,BBB,4.4
...,...,...,...,...,...,...,...,...
53081,2020-01,Y2573F10,304549,Y2573F10,SG9999000020,FLEX LTD,A,6.1
53082,2020-02,Y2573F10,216131,Y2573F10,SG9999000020,FLEX LTD,A,6.9
53083,2020-03,Y2573F10,246969,Y2573F10,SG9999000020,FLEX LTD,A,6.9
53084,2020-04,Y2573F10,411988,Y2573F10,SG9999000020,FLEX LTD,A,6.9


In [85]:
final_table_for_stata.cusip.drop_duplicates().to_excel('cusip_for_monthly_stock_data_msci.xlsx', index=False)

### Load input for monthly stock price data

In [86]:
shrout = pd.read_csv('../data/all_monthly_stock_data_2.csv')

In [87]:
shrout['year_month'] = pd.to_datetime(shrout['MthCalDt'], format='%Y/%m/%d').dt.to_period('M')

In [88]:
shrout.head()

,HdrCUSIP,CUSIP,CUSIP9,MthCalDt,MthPrc,MthCap,ShrOut,year_month
0,68389X10,68389X10,68389X105,2013/12/31,38.2600,"172,070,868.34","4,497,409",2013-12
1,68389X10,68389X10,68389X105,2014/01/31,36.9000,"165,954,392.10","4,497,409",2014-01
2,68389X10,68389X10,68389X105,2014/02/28,39.1100,"175,893,665.99","4,497,409",2014-02
3,68389X10,68389X10,68389X105,2014/03/31,40.9100,"182,413,026.26","4,458,886",2014-03
4,68389X10,68389X10,68389X105,2014/04/30,40.8800,"182,279,259.68","4,458,886",2014-04


In [89]:
required_shrout = shrout.loc[:, ['CUSIP', 'CUSIP9', 'year_month', 'MthPrc', 'MthCap', 'ShrOut']]

In [90]:
required_shrout.isna().sum()

CUSIP           0
CUSIP9          0
year_month      0
MthPrc          0
MthCap          0
ShrOut        152
dtype: int64

In [91]:
required_shrout = required_shrout.dropna(subset='ShrOut')

In [92]:
final_table_stata_and_shrout = final_table_for_stata.merge(required_shrout, how='inner', left_on=['cusip', 'year_month'], right_on=['CUSIP', 'year_month'])

In [93]:
# got data for all 911 firms

In [94]:
final_table_stata_and_shrout.cusip.drop_duplicates()

0        00101J10
29       00105510
114      00108410
174      00120410
181      00123Q10
           ...   
50796    N5374510
50881    N6596X10
50940    N7248212
50979    V7780T10
51064    Y2573F10
Name: cusip, Length: 890, dtype: object

In [95]:
esg_grade_changes_dict_map = {'AAA':7, 
                              'AA':6, 
                              'A':5,
                              'BBB':4,
                             'BB':3,
                             'B':2,
                             'CCC':1}
final_table_stata_and_shrout['esg_grade_num'] = final_table_stata_and_shrout['Company Rating'].map(esg_grade_changes_dict_map)

In [96]:
# source: https://stackoverflow.com/questions/48673046/get-index-where-value-changes-in-pandas-dataframe-column
changes = {}

for col in ['Company Rating']:
    changes[col] = [0] + [idx for idx, (i, j) in enumerate(zip(final_table_stata_and_shrout[col], final_table_stata_and_shrout[col][1:]), 1) if i != j]

for col in ['esg_grade_num']:
    changes['upgrade'] = [idx for idx, (i, j) in enumerate(zip(final_table_stata_and_shrout[col], final_table_stata_and_shrout[col][1:]), 1) if i < j] 

for col in ['esg_grade_num']:
    changes['downgrade'] = [idx for idx, (i, j) in enumerate(zip(final_table_stata_and_shrout[col], final_table_stata_and_shrout[col][1:]), 1) if i > j]        

In [97]:
upgrade_frame = pd.DataFrame({x:'upgrade' for x in changes['upgrade']}, index=[0]).T
downgrade_frame = pd.DataFrame({x:'downgrade' for x in changes['downgrade']}, index=[0]).T
upgrades_and_downgrades = pd.concat([upgrade_frame,downgrade_frame]).rename(columns={0:'up_or_down'})
upgrades_and_downgrades.loc[upgrades_and_downgrades['up_or_down']=='upgrade', 'upgrade_dummy'] = 1
upgrades_and_downgrades.loc[upgrades_and_downgrades['up_or_down']=='downgrade', 'downgrade_dummy'] = 1

In [98]:
upgrades_and_downgrades['upgrade_dummy'] = upgrades_and_downgrades['upgrade_dummy'].fillna(0).astype(int)
upgrades_and_downgrades['downgrade_dummy'] = upgrades_and_downgrades['downgrade_dummy'].fillna(0).astype(int)
upgrades_and_downgrades

,up_or_down,upgrade_dummy,downgrade_dummy
28,upgrade,1,0
98,upgrade,1,0
106,upgrade,1,0
118,upgrade,1,0
232,upgrade,1,0
...,...,...,...
50979,downgrade,0,1
51001,downgrade,0,1
51060,downgrade,0,1
51069,downgrade,0,1


In [99]:
# obs where esg_grade changes
esg_grade_changes = final_table_stata_and_shrout.iloc[changes['Company Rating']].copy()

In [100]:
esg_grade_changes = pd.concat([esg_grade_changes, upgrades_and_downgrades], axis=1)

In [101]:
esg_grade_changes

,year_month,cusip,nbr_shares,cusip_8,ISIN,Asset Name,Company Rating,Industry Adjusted Score,CUSIP,CUSIP9,MthPrc,MthCap,ShrOut,esg_grade_num,up_or_down,upgrade_dummy,downgrade_dummy
0,2013-12,00101J10,231801,00101J10,US00101J1060,ADT CORP,BBB,4.4,00101J10,00101J106,40.4700,"8,165,389.08","201,764",4,NaN,NaN,NaN
28,2016-04,00101J10,42976,00101J10,US00101J1060,ADT CORP,A,7.0,00101J10,00101J106,41.9800,"6,949,621.08","165,546",5,upgrade,1.0,0.0
29,2013-12,00105510,487789,00105510,US0010551028,AFLAC INC,BBB,4.3,00105510,001055102,66.8000,"31,134,812.00","466,090",4,downgrade,0.0,1.0
64,2016-11,00105510,226799,00105510,US0010551028,AFLAC INC,BB,3.4,00105510,001055102,71.3800,"29,130,320.76","408,102",3,downgrade,0.0,1.0
82,2018-05,00105510,555557,00105510,US0010551028,AFLAC INC,B,2.7,00105510,001055102,45.0600,"34,883,965.02","774,167",2,downgrade,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51060,2020-09,V7780T10,252792,V7780T10,LR0008862868,ROYAL CARIBBEAN GROUP,BB,4.0,V7780T10,V7780T103,64.7300,"13,895,459.64","214,668",3,downgrade,0.0,1.0
51064,2013-12,Y2573F10,7732,Y2573F10,SG9999000020,FLEX LTD,AA,8.0,Y2573F10,Y2573F102,7.7700,"4,679,319.33","602,229",6,upgrade,1.0,0.0
51069,2014-05,Y2573F10,31407,Y2573F10,SG9999000020,FLEX LTD,A,6.1,Y2573F10,Y2573F102,10.1700,"5,954,097.69","585,457",5,downgrade,0.0,1.0
51080,2015-04,Y2573F10,66298,Y2573F10,SG9999000020,FLEX LTD,AA,8.4,Y2573F10,Y2573F102,11.5250,"6,492,297.58","563,323",6,upgrade,1.0,0.0


In [102]:
# concat treatment dates into final table

In [103]:
final_table_stata_and_shrout = final_table_stata_and_shrout.drop(['CUSIP', 'CUSIP9'], axis=1)

### Shares outstanding (shrout) is in thousands
* https://www.crsp.org/files/data_descriptions_guide_0.pdf

In [104]:
final_table_stata_and_shrout['ShrOut'] = final_table_stata_and_shrout['ShrOut'].apply(lambda x: x.replace(',', ''))

In [105]:
final_table_stata_and_shrout = final_table_stata_and_shrout.apply(pd.to_numeric, errors='ignore')

In [106]:
final_table_stata_and_shrout['esg_ownership'] = final_table_stata_and_shrout['nbr_shares']/(final_table_stata_and_shrout['ShrOut']*1000)
final_table_stata_and_shrout['esg_ownership_(%)'] = final_table_stata_and_shrout['esg_ownership']*100

In [107]:
final_table_stata_and_shrout

,year_month,cusip,nbr_shares,cusip_8,ISIN,Asset Name,Company Rating,Industry Adjusted Score,MthPrc,MthCap,ShrOut,esg_grade_num,esg_ownership,esg_ownership_(%)
0,2013-12,00101J10,231801,00101J10,US00101J1060,ADT CORP,BBB,4.4,40.4700,"8,165,389.08",201764,4,0.001149,0.114887
1,2014-01,00101J10,152417,00101J10,US00101J1060,ADT CORP,BBB,4.4,30.0400,"5,506,632.40",183310,4,0.000831,0.083147
2,2014-02,00101J10,136968,00101J10,US00101J1060,ADT CORP,BBB,4.4,30.7100,"5,629,450.10",183310,4,0.000747,0.074719
3,2014-03,00101J10,59068,00101J10,US00101J1060,ADT CORP,BBB,4.4,29.9500,"5,490,134.50",183310,4,0.000322,0.032223
4,2014-04,00101J10,17834,00101J10,US00101J1060,ADT CORP,BBB,4.4,30.2400,"5,269,199.04",174246,4,0.000102,0.010235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51137,2020-01,Y2573F10,304549,Y2573F10,SG9999000020,FLEX LTD,A,6.1,13.1500,"6,627,494.80",503992,5,0.000604,0.060427
51138,2020-02,Y2573F10,216131,Y2573F10,SG9999000020,FLEX LTD,A,6.9,11.1100,"5,599,351.12",503992,5,0.000429,0.042884
51139,2020-03,Y2573F10,246969,Y2573F10,SG9999000020,FLEX LTD,A,6.9,8.3750,"4,165,942.75",497426,5,0.000496,0.049649
51140,2020-04,Y2573F10,411988,Y2573F10,SG9999000020,FLEX LTD,A,6.9,9.7600,"4,854,877.76",497426,5,0.000828,0.082824


In [108]:
final_table_stata_and_shrout.describe().T.apply(lambda x: x.map('{:.2f}'.format))

,count,mean,std,min,25%,50%,75%,max
nbr_shares,51142.00,338886.47,664870.27,1.00,23929.00,99558.50,329604.50,11356920.00
Industry Adjusted Score,51142.00,4.83,2.18,0.00,3.20,4.70,6.40,10.00
ShrOut,51142.00,543367.94,943234.79,3590.00,140745.00,267215.00,527198.00,17102536.00
esg_grade_num,51142.00,3.85,1.46,1.00,3.00,4.00,5.00,7.00
esg_ownership,51142.00,0.00,0.00,0.00,0.00,0.00,0.00,0.02
esg_ownership_(%),51142.00,0.08,0.13,0.00,0.01,0.04,0.09,1.94


In [109]:
esg_grade_changes = esg_grade_changes.rename(columns={'year_month': 'treatment_date', 'cusip':'cusip_grade_change'})

In [110]:
# set first month of each security as untreated.
esg_grade_changes.loc[esg_grade_changes.drop_duplicates('cusip_grade_change').index, ['up_or_down', 'upgrade_dummy', 'downgrade_dummy']] = 0

In [111]:
esg_grade_changes.loc[esg_grade_changes['up_or_down']==0, 'treatment_date']= np.nan

In [112]:
esg_grade_changes = esg_grade_changes.loc[:, ['treatment_date', 'up_or_down', 'upgrade_dummy', 'downgrade_dummy']]

In [113]:
final_table_stata_and_shrout_all_changes = pd.concat([final_table_stata_and_shrout, esg_grade_changes], axis=1)

In [114]:
first_upgrade_index = final_table_stata_and_shrout_all_changes[final_table_stata_and_shrout_all_changes['up_or_down']=='upgrade'].drop_duplicates(subset=['cusip', 'up_or_down']).index

In [115]:
first_downgrade_index = final_table_stata_and_shrout_all_changes[final_table_stata_and_shrout_all_changes['up_or_down']=='downgrade'].drop_duplicates(subset=['cusip', 'up_or_down']).index

In [116]:
first_up_and_downgrade_list = list(first_upgrade_index) + list(first_downgrade_index)

In [117]:
first_up_and_downgrade_table = esg_grade_changes.loc[first_up_and_downgrade_list]

In [118]:
final_table_stata_and_shrout = pd.concat([final_table_stata_and_shrout, first_up_and_downgrade_table], axis=1)

In [119]:
final_table_stata_and_shrout.up_or_down = final_table_stata_and_shrout.up_or_down.fillna(0)
final_table_stata_and_shrout.upgrade_dummy = final_table_stata_and_shrout.upgrade_dummy.fillna(0).astype(int)
final_table_stata_and_shrout.downgrade_dummy = final_table_stata_and_shrout.downgrade_dummy.fillna(0).astype(int)

In [120]:
# here we only look at the first upgrade and downgrades (i.e. we only consider the initial treatment for each security)
# we do not consider multiple upgrades/downgrades for same security
final_table_stata_and_shrout.groupby(['cusip'])['up_or_down'].value_counts().to_frame().head(10)

up_or_down
cusip    up_or_down            
00101J10 0                   28
         upgrade              1
00105510 0                   83
         downgrade            1
         upgrade              1
00108410 0                   59
         downgrade            1
00120410 0                    7
00123Q10 0                   51
00130H10 0                   83

In [121]:
id_for_cusips_dict = {x:i for i,x in enumerate(final_table_stata_and_shrout.drop_duplicates('cusip').cusip, start=1)}

In [122]:
final_table_stata_and_shrout['ID'] = final_table_stata_and_shrout.cusip.map(id_for_cusips_dict)

In [123]:
final_table_stata_and_shrout.year_month = final_table_stata_and_shrout.year_month.dt.to_timestamp()
final_table_stata_and_shrout.treatment_date = final_table_stata_and_shrout.treatment_date.dt.to_timestamp()

In [124]:
final_table_stata_and_shrout = final_table_stata_and_shrout.drop('cusip_8', axis=1)

In [125]:
final_table_stata_and_shrout = final_table_stata_and_shrout.set_index(['ID', 'year_month']).reset_index()

In [126]:
final_table_stata_and_shrout = final_table_stata_and_shrout.drop_duplicates(subset=['ID', 'year_month'])

In [127]:
final_table_stata_and_shrout.shape

(51036, 18)

In [128]:
final_table_stata_and_shrout.up_or_down[final_table_stata_and_shrout.up_or_down==0] = None

In [129]:
# fill dates for upgrades and downgrades into two columns

In [130]:
treatment_dict_upgrade = (final_table_stata_and_shrout.drop_duplicates(['cusip', 'treatment_date']).set_index('up_or_down')
 .loc['upgrade',['cusip','treatment_date']]
 .reset_index()
 .set_index('cusip')
 .treatment_date
 .to_dict())

In [131]:
treatment_dict_downgrade = (final_table_stata_and_shrout.drop_duplicates(['cusip', 'treatment_date']).set_index('up_or_down')
 .loc['downgrade',['cusip','treatment_date']]
 .reset_index()
 .set_index('cusip')
 .treatment_date
 .to_dict())

In [132]:
final_table_stata_and_shrout['treatment_date_up'] = final_table_stata_and_shrout.cusip.map(treatment_dict_upgrade)

In [133]:
final_table_stata_and_shrout['treatment_date_down'] = final_table_stata_and_shrout.cusip.map(treatment_dict_downgrade)

In [134]:
# never treated units
never_treated_units = (final_table_stata_and_shrout[(final_table_stata_and_shrout.treatment_date_up.isna())
                              & (final_table_stata_and_shrout.treatment_date_down.isna())]
 .drop_duplicates(subset='cusip'))

In [135]:
final_table_stata_and_shrout = final_table_stata_and_shrout.rename(columns={
    'Company Rating':'company_rating', 
    'Industry Adjusted Score': 'industry_adjusted_score',
    'Asset Name':'asset_name',
    'esg_ownership_(%)': 'esg_ownership_percent'})

In [136]:
final_table_stata_and_shrout.to_stata('final_table_stata_and_shrout_msci_excluding_index_etf.dta', write_index=False, convert_dates={'year_month':'tm', 'treatment_date':'tm', 'treatment_date_up':'tm', 'treatment_date_down':'tm'})

In [137]:
final_table_stata_and_shrout.to_csv('final_table_stata_and_shrout_msci_excluding_index_etf.csv', index=False)

### Trimming esg_ownership variable at 1st and 99th percentiles for each month to remove outliers.

In [138]:
%%time
df_trimmed_for_outliers = []
for x in final_table_stata_and_shrout.year_month.drop_duplicates():
    df_trimmed_for_outliers.append(final_table_stata_and_shrout[(final_table_stata_and_shrout['year_month']==x)
                            & (final_table_stata_and_shrout['esg_ownership']<final_table_stata_and_shrout['esg_ownership'].quantile(0.99))
                            & (final_table_stata_and_shrout['esg_ownership']>final_table_stata_and_shrout['esg_ownership'].quantile(0.01))])

CPU times: total: 1.14 s
Wall time: 1.32 s


In [139]:
%%time
final_table_trimmed_for_outliers = pd.concat(df_trimmed_for_outliers)

CPU times: total: 172 ms
Wall time: 235 ms


In [140]:
final_table_trimmed_for_outliers.to_stata('final_table_trimmed_for_outliers_msci_excluding_index_etf.dta', write_index=False, convert_dates={'year_month':'tm', 'treatment_date':'tm', 'treatment_date_up':'tm', 'treatment_date_down':'tm'})